# Example of BOMD MD (NVE, NVT)

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   402.89   2.083079e-01 -1.357583e+00 -1.149275e+00 ||    458.27   2.369437e-01 1.399904e+00 1.636847e+00 || 
     2   384.41   1.987577e-01 -1.347987e+00 -1.149230e+00 ||    534.63   2.764233e-01 1.360007e+00 1.636430e+00 || 
     3   357.74   1.849660e-01 -1.334125e+00 -1.149159e+00 ||    612.47   3.166691e-01 1.319273e+00 1.635942e+00 || 
     4   325.26   1.681745e-01 -1.317248e+00 -1.149073e+00 ||    677.87   3.504868e-01 1.284974e+00 1.635460e+00 || 
     5   289.80   1.498360e-01 -1.298822e+00 -1.148986e+00 ||    717.49   3.709717e-01 1.264111e+00 1.635083e+00 || 
     6   254.14   1.314024e-01 -1.280313e+00 -1.148911e+00 ||    721.43   3.730103e-01 1.261900e+00 1.634910e+00 || 
     7   220.71   1.141176e-01 -1.262973e+00 -1.148855e+00 ||    686.11   3.547478e-01 1.280262e+00 1.635009e+00 || 
     8   191.21   9.886069e-02 -1.247686e+00 -1.148825e+00 ||    616.02   3.185053e-01 1.31687

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [4]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_E_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   459.48   2.375697e-01 -1.065373e-01 1.310324e-01 ||    168.05   8.688837e-02 2.185935e+00 2.272824e+00 || 
     2   518.75   2.682136e-01 -1.371812e-01 1.310324e-01 ||    178.53   9.230918e-02 2.180515e+00 2.272824e+00 || 
     3   619.35   3.202296e-01 -1.891973e-01 1.310324e-01 ||    274.63   1.419926e-01 2.130831e+00 2.272824e+00 || 
     4   768.54   3.973655e-01 -2.663331e-01 1.310324e-01 ||    449.57   2.324442e-01 2.040380e+00 2.272824e+00 || 
     5   969.37   5.012008e-01 -3.701685e-01 1.310324e-01 ||    690.60   3.570699e-01 1.915754e+00 2.272824e+00 || 
     6  1219.11   6.303263e-01 -4.992939e-01 1.310324e-01 ||    978.60   5.059744e-01 1.766849e+00 2.272824e+00 || 
     7  1508.31   7.798558e-01 -6.488234e-01 1.310324e-01 ||   1287.59   6.657347e-01 1.607089e+00 2.272824e+00 || 
     8  1820.41   9.412238e-01 -8.101914e-01 1.310324e-01 ||   1584.47   8.192327e-01 1.453591e+00 2.

## Scale velocities to control temperature

In [5]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Basic_T_cntrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -1.122726e-01 9.454342e-02 ||    400.00   2.068160e-01 2.129391e+00 2.336207e+00 || 
     2   400.00   2.068160e-01 -1.419430e-01 6.487300e-02 ||    400.00   2.068160e-01 2.070398e+00 2.277214e+00 || 
     3   400.00   2.068160e-01 -1.932226e-01 1.359339e-02 ||    400.00   2.068160e-01 1.982957e+00 2.189773e+00 || 
     4   400.00   2.068160e-01 -2.653099e-01 -5.849388e-02 ||    400.00   2.068160e-01 1.879045e+00 2.085861e+00 || 
     5   400.00   2.068160e-01 -3.546723e-01 -1.478563e-01 ||    400.00   2.068160e-01 1.768744e+00 1.975560e+00 || 
     6   400.00   2.068160e-01 -4.560043e-01 -2.491883e-01 ||    400.00   2.068160e-01 1.659386e+00 1.866202e+00 || 
     7   400.00   2.068160e-01 -5.636089e-01 -3.567929e-01 ||    400.00   2.068160e-01 1.555986e+00 1.762802e+00 || 
     8   400.00   2.068160e-01 -6.724048e-01 -4.655888e-01 ||    400.00   2.068160e-01 1.461955e+

## Langevin Thermostat

In [6]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/3_MD_BOMD_Langevin'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# specify damping factor
md =  Molecular_Dynamics_Langevin(damp = 100.0, seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   498.76   2.578773e-01 -1.359442e+00 -1.101565e+00 ||    524.55   2.712154e-01 1.383468e+00 1.654683e+00 || 
     2   496.28   2.565966e-01 -1.352851e+00 -1.096255e+00 ||    623.87   3.225650e-01 1.331952e+00 1.654517e+00 || 
     3   442.75   2.289208e-01 -1.341806e+00 -1.112885e+00 ||    715.72   3.700540e-01 1.288316e+00 1.658370e+00 || 
     4   397.35   2.054468e-01 -1.327693e+00 -1.122246e+00 ||    766.27   3.961902e-01 1.261110e+00 1.657300e+00 || 
     5   372.56   1.926280e-01 -1.310358e+00 -1.117730e+00 ||    726.77   3.757682e-01 1.255937e+00 1.631705e+00 || 
     6   337.42   1.744580e-01 -1.290099e+00 -1.115641e+00 ||    637.08   3.293955e-01 1.274611e+00 1.604007e+00 || 
     7   309.88   1.602214e-01 -1.268829e+00 -1.108607e+00 ||    546.38   2.825016e-01 1.311417e+00 1.593919e+00 || 
     8   275.01   1.421932e-01 -1.245584e+00 -1.103391e+00 ||    457.86   2.367295e-01 1.35944